- 해당 프로그램 코딩은 임형욱이 개발 했으며 사용자 허락 없이 절대 상업용으로 사용해선 안됩니다.
- 또한 이 프로그램으로 특정 회사에 피해를 줄 경우
- 모든 책임은 프로그램 사용자에게 있음을 미리 공지합니다.
- 이 프로그램과 관련한 문의는 hwlim2010@naver.com으로 보내주세요^^

In [1]:
# 경남도민일보 사건사고 기사 크롤링하기
# http://www.idomin.com/

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import math
import numpy 
import pandas as pd  
import xlwt
import random
import os   # 폴더 생성하기 위해 필요함
import requests
import re

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print("경남도민일보 사건사고 기사 크롤링하기")
print("=" *100)

query_txt = '경남도민일보 사건사고' # 파일 저장 이름용 변수

cnt = int(input('1.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 20) # 한 페이지에 10-건의 기사가 보여짐
print("크롤링 할 총 페이지 번호: ",page_cnt)

#Step 3. 수집된 데이터를 저장할 파일 이름 입력받기 ★★★★★★
f_dir = input("2.파일을 저장할 폴더명만 쓰세요(사용자 에게 맞는 절대경로 입력하세요.):")
if f_dir == '' :
    f_dir="c:\\Users\\"

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'
    
#Step 4. 크롬 드라이버 설정 및 웹 페이지 열기
chrome_path = "C:/Users/pc/chromedriver/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

driver.get('http://www.idomin.com/')
#driver.maximize_window() 
time.sleep(2)

#Step 5. 자동으로 검색어 입력 후 조회하기
driver.find_element_by_class_name('nav-opener').click()
time.sleep(2)
driver.find_element_by_link_text('사회').click()
time.sleep(2)
driver.find_element_by_link_text('사건·사고').click()
time.sleep(2)

#Step 7. 각 뉴스기사의 상세 정보를 수집합니다

click_count = 1

# 각 뉴스기사의 URL 주소를 추출하여 목록을 먼저 만든 후 상세 크롤링을 진행합니다.
# URL 수집하여 목록 만들기
url_list=[]
count = 0
#while ( click_count <= page_cnt + 1)  :
while page_cnt:
    try:
            더보기 = driver.find_element_by_link_text("더보기").click()
            더보기.click()
            time.sleep(1)
    except:
        break
        
print("기사의 URL 정보를 추출합니다==============")
time.sleep(2)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

result_1 = soup.find('div','article-list for-custom')
result_url_list=result_1.find('section','cus-section').find_all('div','items')

for i in result_url_list:
    url = i.find('a')['href']
    print(url)
    full_url = 'http://www.idomin.com/'+ url
    url_list.append(full_url)

    count += 1

    if count == cnt :
        break  

print("기사 수집 완료 ====================")
print("\n")
    
# Step 8. 각 뉴스 별 상세 내역을 수집합니다
no = 1

title2=[]
date2=[]
content2=[]

for i in range(0,len(url_list)):  
            
    print("\n")
    print("%s 번째 경남도민일보 사건사고 기사 상세 정보입니다==========================" %no)

    driver.get(url_list[i])

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    content_all = soup.find('div','user-content')

    # 뉴스제목 추출
    title = content_all.find('div','article-head-title').get_text()
    print('1.뉴스제목:' , title)
    title2.append(title)

    # 뉴스작성일
    e_date = content_all.find('div','info-text fixed').find('ul','no-bullet auto-marbtm-0 line-height-6').find_all('li')
    e_date1=e_date[1].get_text()
    print('2.뉴스작성일: ',e_date1.split(' ')[3])
    date2.append(e_date1.split(' ')[3])
    
    # 뉴스내용
    content = str(content_all.find('article','article-veiw-body view-page font-size17').find_all('p'))
    content1 = re.sub('<.+?>', '', content, 0).strip()
    print('3.뉴스내용: ',content1)
    content2.append(content1)
    
    #driver.back()  # 이전 페이지로 돌아가기 ★★★★★
    
    time.sleep(2)
    
    no += 1

    if no > cnt :
        break

# Step 9. 출력 결과를 저장하기
# 출력 결과를 표(데이터 프레임) 형태로 만들기

gnnews = pd.DataFrame()

gnnews['뉴스작성일'] = pd.Series(date2)
gnnews['뉴스제목'] = pd.Series(title2) 
gnnews['뉴스내용'] = pd.Series(content2)
gnnews['뉴스URL'] = pd.Series(url_list)
       
# csv 형태로 저장하기
gnnews.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
gnnews.to_excel(fx_name , index=False)

driver.close( )


경남도민일보 사건사고 기사 크롤링하기
1.크롤링 할 건수는 몇건입니까?: 50
크롤링 할 총 페이지 번호:  3
2.파일을 저장할 폴더명만 쓰세요(사용자 에게 맞는 절대경로 입력하세요.):
기사의 URL 정보를 추출합니다==============
/news/articleView.html?idxno=770535
/news/articleView.html?idxno=770428
/news/articleView.html?idxno=770422
/news/articleView.html?idxno=770269
/news/articleView.html?idxno=769741
/news/articleView.html?idxno=769649
/news/articleView.html?idxno=769600
/news/articleView.html?idxno=768999
/news/articleView.html?idxno=768336
/news/articleView.html?idxno=768194
/news/articleView.html?idxno=767587
/news/articleView.html?idxno=767483
/news/articleView.html?idxno=767439
/news/articleView.html?idxno=767322
/news/articleView.html?idxno=767230
/news/articleView.html?idxno=767164
/news/articleView.html?idxno=767158
/news/articleView.html?idxno=767156
/news/articleView.html?idxno=767037
/news/articleView.html?idxno=767036
/news/articleView.html?idxno=767034&page=2&total=12708
/news/articleView.html?idxno=766957&page=2&total=12708
/news/articleView.html?idxno=766



11 번째 경남도민일보 사건사고 기사 상세 정보입니다==========================
1.뉴스제목: 창원해경 해상 응급환자 긴급 이송
2.뉴스작성일:  2021-07-20
3.뉴스내용:  [양경찰서는 19일 오후 2시 17분께 창원시 진해구 잠도 동방 1.2해리 인근 해상에서 작업하던 모래운반선(1500t급) 내 응급환자를 급히 이송했다고 밝혔다., 창원해경은 모래운반선에서 작업을 하던 선원 ㄱ 씨가 갑자기 의식을 잃고 쓰러지면서 바닥에 부딪히는 바람에 통증을 호소했다고 설명했다. 모래운반선 선장이 마산VTS(해상교통관제)를 거쳐 해경에 신고했고, 창원해경에서 경비함정을 현장으로 급히 보냈다. 창원해경은 응급처치를 마치고 창원시 마산합포구 구산면 원전항에 대기하던 119 구급차량에 환자를 넘겼다고 밝혔다.,  ,  , 시민사회부 소속으로 창원중부경찰서를 출입하면서 노동 분야를 맡고 있습니다.
제보는 010-8994-4329]


12 번째 경남도민일보 사건사고 기사 상세 정보입니다==========================
1.뉴스제목: 양산 코로나19 생활치료센터서 불
2.뉴스작성일:  2021-07-19
3.뉴스내용:  [코로나19 생활치료센터로 운영 중인 양산 에덴벨리 리조트에서 불이 나 이곳에서 치료를 받던 확진자와 자가격리자가 대피하는 소동이 벌어졌다. , 19일 오후 8시 45분께 리조트 9층에서 시작한 불로 확진자와 자가격리자 등 40여 명이 긴급대피했다. 현재 이곳에는 250여 명이 치료를 받고 있으며, 불이 난 9층을 제외한 나머지 인원은 따로 대피하지 않은 채 신고를 받고 출동한 소방당국이 오후 9시 12분께 화재를 진압할 때까지 방에서 대기한 것으로 알려졌다. 다행히 인명피해는 없었다. , 소방당국은 에어컨 과열로 실외기에서 불이 붙은 것으로 추정하는 가운데 정확한 원인과 피해 규모를 조사하고 있다., 경남도민일보 기자]


13 번째 경남도민일보 사건사고 기사 상세 정보입니다========================

1.뉴스제목: 폭우에 창원 마산합포구 한 도로에 산사태
2.뉴스작성일:  2021-07-07
3.뉴스내용:  [7일 저녁 갑자기 쏟아진 폭우에 창원시 마산합포구 한 도로에 산사태가 일어났다. 다행히 인명피해는 없었지만 현재 통행이 불가능한 상황이다., 창원소방본부는 7일 오후 6시 44분 창원시 마산합포구 진전면 회진로(창포리 산 34-3번지 지점) 비탈면 토사가 쏟아져 내리는 사고가 났다고 밝혔다. 이 사고로 차량 1대가 보호난간(가드레일)을 뚫고 도로 아래로 떨어졌지만, 운전자는 큰 부상 없이 대피했다., 소방당국은 인명피해 유무를 확인한 뒤, 현장을 마산합포구청에 인계했다. 구청은 현재 토사가 흘러내린 현장을 굴삭기로 수습 중이다. 동진교와 불과 100m 떨어진 이 지점은 현재 왕복 2차선이 모두 막혀 통행이 불가능하다,  , 사건, 환경, 이주민 분야를 담당하고 있습니다. 그 외에도 다양한 제보 기다리겠습니다. 010-9238-2568]


28 번째 경남도민일보 사건사고 기사 상세 정보입니다==========================
1.뉴스제목: 앗! 벼락 맞은 신호등
2.뉴스작성일:  2021-07-07
3.뉴스내용:  [천둥 번개를 동반한 폭우가 쏟아진 7일 오후 7시께, 김해시 장유2동 부영7단지 앞 네거리 신호등이 낙뢰 피해로 파손되면서 경찰이 긴급 교통정리를 하고 있다. ,  ,  , 돌아온 탕아?
1년간의 본사 내근을 끝내고 다시 스포츠로 돌아왔습니다. 경남FC를 비롯해 프로 종목은 물론 다양한 전문체육과 생활체육 전반들 다룹니다.]


29 번째 경남도민일보 사건사고 기사 상세 정보입니다==========================
1.뉴스제목: 사천 삼천포신항서 트럭 추락
2.뉴스작성일:  2021-07-07
3.뉴스내용:  [　
사천시 삼천포신항에 주차돼 있던 1.5t 트럭이 바다에 빠지는 사고가 났다.
통영해양경찰서는 7일 오전 8시 48분께 삼천포신항에서 차량 소유자가 주차하고 내린 후 잠시 뒤 차량이 움직여 해상에 추락했다

<ipython-input-1-61cc83e6759b>:158: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  gnnews.to_excel(fx_name , index=False)
